
## Agenten
KI Agenten sind laut vielen Experten das nächste große Ding in Sachen KIs. Ganz unabhängig davon, ob das stimmt, können (autonome) KI Agenten hilfreich sein und es lohnt sich, sich damit auseinander zu setzen.
Meiner Einschätzung nach erweitern KI Agenten das Spektrum an Werkzeugen, die man zum Lösen von Aufgaben benötigt. Diese ermöglichen auch Aufgaben zu löschen, die zuvor eigentlich nicht automatisierbar waren.

Dennoch, bei aller Euphorie, muss man sagen, dass Stand jetzt autonome Agenten noch in den Kinderschuhen stecken und häufig Fehler begehen. Komplexe Aufgaben sind also noch in weiter Ferne.

Heute schauen wir uns an, wie man Agenten erstellt und wozu sie gut sein könnten. Wir beginnen mit sehr einfachen Formen und steigern uns dann und setzen das Agentenframework LangGraph ein.


## Was sind denn Agenten überhaupt?

Die Definitionen, was ein Agent ist, gehen wie so oft stark auseinander. Wo sich die meisten jedoch einig sind, ist dass wenn man der KI Werkzeuge zur Verfügung stellt und die KI selbst entscheidet, wann sie diese Werkzeuge einsetzen möchte, die erste Stufe eines Agenten erreicht ist.
Es gibt aber je nach Definition auch Agenten ohne Werkzeuge, also z.B. Agenten, die durch Prompting-Techniken erzeugt werden. 

Meist weist man einem Agenten eine Rolle zu, z.B. Organisator, Qualitätssicherer, Coder usw. Der Hintergrund ist der, dass Sprachmodelle häufig besser performen, wenn man sie in eine Rolle versetzt. In Agentensystemen, hat man also meist ein paar Agenten mit unterschiedlichen Rollen.
In diesen Systemen tauschen die Agenten Daten aus, sei es dadurch, dass sie einen Prompt mit allen notwenigen Informationen bekommen und die Antwort wird z.B. einen anderen Agenten übergeben, oder z.B. durch einen gemeinsamen Speicher, auf den jeder Agent Zugriff hat.

Gibt man einer KI die Möglichkeit Werkzeuge zu nutzen, läuft das häufig so ab, dass die KI die Ausführung anfragt und jemand oder etwas führt das Werkzeug aus und übergibt das Ergebnis wiederum an die KI, die dann damit arbeiten kann. D.h. die KI kann nicht einfach selbst etwas ausführen. Allerings bieten Agentensysteme die Möglichkeit an, dass diese Tool-Anfragen automatisch ausgeführt werden, ohne dass ein Mensch die Ausführung genehmigen muss oder gar selbst ausführen muss. Hier sprechen wir dann von *autonomen* Agenten. Häufig kombiniert man das aber mit _Human-In-The-Loop_ Ansätzen. D.h. bei kritischen Ausführungen wird ein Mensch erstmal um Erlaubnis gebeten. Auch wird das genutzt um z.B. zu entscheiden, wie etwas weitergehen soll, im Sinne von entweder Weg A, B oder C z.B. Oder um weitere Informationen abzufragen.

Bei Systemen mit mehreren Agenten kann man für jeden Agent unterschiedliche Modelle verwenden. So kann man für jede Aufgabe das beste Modell auswählen. Während z.B. der Organisator-Agent ein Modell einsetzt, das gut logisch kombinieren kann, setzt der Coder ein Modell ein, das besonders gut Code produzieren kann.




# Erster Agent

Es ist an der Zeit einen Agenten zu erstellen. Und zwar werden wir einen sogenannten ReAct Agenten mit einem Tool erstellen. Was ReAct ist, sehen wir gleich. Als Tool möchten wir dem Agenten Zugriff auf Wikipedia geben, damit der Agent mithilfe der Wikipedia-Artikel fundierte Antworten geben kann.

## ReAct

Ein _ReAct Agent_ ist ein KI-System, das "Reasoning and Acting" (Denken und Handeln) kombiniert.

- Abwechslung von Denken und Handeln: Der Agent wechselt zwischen Denkphasen, in denen er plant und schlussfolgert, und Handlungsphasen, in denen er Aktionen ausführt.
- Schrittweise Problemlösung: Komplexe Aufgaben werden in kleinere Schritte zerlegt und sequentiell bearbeitet.
- Anpassungsfähigkeit: Der Agent kann seinen Ansatz basierend auf Zwischenergebnissen oder neuen Informationen anpassen.
- Werkzeuge: ReAct Agenten können oft externe Tools oder Datenquellen in ihren Problemlösungsprozess einbinden.

## Tools

Wie funktioniert das mit den Tools? Die KI, also das Sprachmodell, kann nicht selbst Funktionen oder ähnliches ausführen. Aber die KI kann darum bitten.

Wenn man der KI also sagt, dass ihr Tools zur Verfügung stehen und man außerdem beschreibt, welche Argumente die Tools benötigen, dann kann die KI sich dafür entscheiden. In der Tat werden die Modelle darauf trainiert. Der Chat-Request an Chat-GPT (also die direkte OpenAI-Schnittstelle) enthält sogar einen besonderen Bereich außerhalb des Prompts für die Tools. Man übergibt dabei einen Namen des Tools und ganz wichtig eine Beschreibung, wofür das Tool gut sein soll. Die Beschreibung dient der KI dazu zu verstehen, was das Tool kann. Dann listet man die erwarteten Parameter auf. Auch hier mit Namen, Datentyp und Beschreibung. Und klar, die Beschreibung erklärt der KI wozu der Parameter gut ist.


Das Ganze können wir mal kurz über einen simplen Prompt simulieren. Nehmen wir das allererste Beispiel, das wir programmiert haben und passen es an.

In [ ]:
!pip install langchain langchain_openai python-dotenv

In [13]:
import os
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-4o")

messages = [
    SystemMessage("Du bist ein hilfreicher Chatbot. Falls du nach der Uhrzeit gefragt wirst, dann kannst du ein Tool namens 'datetime' verwenden."
                  "Wenn du das Tool brauchst, dann antworte ausschließlich mit 'datetime'."),
    HumanMessage("Wieviel Uhr ist es?"),
]

response_text = model.invoke(messages)

if response_text.content == "datetime":
    import datetime
    current_time = datetime.datetime.now().strftime("%d-%m-%Y %H:%M")

    messages.append(AIMessage(f"Antwort vom Tool ist: {current_time}. Beantworte jetzt die Frage des Nutzers"))

    response_text = model.invoke(messages)

response_text.content



'Es ist 22:09 Uhr.'